In [2]:
import itertools as its
import pandas as pd
from collections import Counter

In [3]:
trades = pd.read_csv('data/NEPH-Trades.csv')
marketMaker = pd.read_csv('data/market-maker.csv')

In [4]:
sizes = trades['SIZE'].values

In [5]:
sizes = sizes.astype(int)
counter = Counter(sizes)

In [6]:
for k, v in counter.items():
    counter[k] = min(counter[k], 6)

In [12]:
reduced_size = []
for k,v in counter.items():
    for i in range(v):
        reduced_size.append(k)
reduced_size = sorted(reduced_size)

In [8]:
def combs(trades, orders, size):
    for comb in its.combinations(trades, orders):
        if sum(comb) == size:
            yield comb

In [14]:
indices = {}
for i, v in enumerate(reduced_size):
    indices[v] = max(indices.get(v, -1), i)

In [9]:
gen = combs(reduced_size, 6, 1527)

In [10]:
t = next(gen)

In [11]:
# find "alternates"
def find_new(start, trades, indices):
    ans = [list(start)]
    l,r = 0, len(start) - 1
    while start[l] == start[l + 1]:
        l += 1
    while start[r] == start[r-1]:
        r -= 1

    while l < r:
        # switching l and r
        low_limit = high_limit = (start[l] + start[r])//2
        if r - l > 1:
            low_limit = start[l + 1]
            high_limit = start[r-1]
        # search for numbers between the limits
        left, right = indices[start[l]], indices[start[r]]
        target = start[l] + start[r]
        
        while left < right:
            total = trades[right] + trades[left]
            if total == target:
                candidate = start.copy()
                candidate[l], candidate[r]
                ans.append()
        while start[l + 1] == start[l]:
            l += 1
        while start[r] == start[r-1]:
            r -= 1
    return ans

(100, 100, 100, 282, 450, 495)

In [17]:
def find_swaps(start, trades, target_sum, max_for_first, min_for_second):
    """
    Find all pairs in trades that sum to target_sum, where the first is less than max_for_first
    and the second is greater than or equal to min_for_second.
    """
    ans = []
    left, right = 0, len(trades) - 1
    while left < right:
        current_sum = trades[left] + trades[right]
        if current_sum == target_sum and trades[left] < max_for_first and trades[right] >= min_for_second:
            ans.append((trades[left], trades[right]))
            left += 1
            right -= 1
        elif current_sum < target_sum or trades[left] >= max_for_first:
            left += 1
        else:
            right -= 1
    return ans

def find_new(start, trades, indices):
    ans = []
    # Assuming 'start' is already sorted, we maintain the sorted order in our swaps.
    # We're looking to swap out pairs, so we find the sum of the two elements to replace.
    for i in range(len(start) - 1):
        for j in range(i + 1, len(start)):
            if i == 2 and j == len(start) - 1: # Specific to the example given, could be generalized
                target_sum = start[i] + start[j]
                max_for_first = start[i + 1]
                min_for_second = start[j - 1]
                swaps = find_swaps(start, trades, target_sum, max_for_first, min_for_second)
                for first, second in swaps:
                    new_start = start.copy()
                    new_start[i], new_start[j] = first, second
                    if new_start not in ans:
                        ans.append(new_start)
    return ans

# Example usage
start = [100, 100, 100, 282, 450, 495]
trades = sorted(set(start + [num for num in range(101, 494)])) # Example trades array
indices = {val: idx for idx, val in enumerate(trades)} # Create indices dictionary

new_combinations = find_new(start, trades, indices)
for combo in new_combinations:
    print(combo)


[100, 100, 100, 282, 450, 495]
[100, 100, 102, 282, 450, 493]
[100, 100, 103, 282, 450, 492]
[100, 100, 104, 282, 450, 491]
[100, 100, 105, 282, 450, 490]
[100, 100, 106, 282, 450, 489]
[100, 100, 107, 282, 450, 488]
[100, 100, 108, 282, 450, 487]
[100, 100, 109, 282, 450, 486]
[100, 100, 110, 282, 450, 485]
[100, 100, 111, 282, 450, 484]
[100, 100, 112, 282, 450, 483]
[100, 100, 113, 282, 450, 482]
[100, 100, 114, 282, 450, 481]
[100, 100, 115, 282, 450, 480]
[100, 100, 116, 282, 450, 479]
[100, 100, 117, 282, 450, 478]
[100, 100, 118, 282, 450, 477]
[100, 100, 119, 282, 450, 476]
[100, 100, 120, 282, 450, 475]
[100, 100, 121, 282, 450, 474]
[100, 100, 122, 282, 450, 473]
[100, 100, 123, 282, 450, 472]
[100, 100, 124, 282, 450, 471]
[100, 100, 125, 282, 450, 470]
[100, 100, 126, 282, 450, 469]
[100, 100, 127, 282, 450, 468]
[100, 100, 128, 282, 450, 467]
[100, 100, 129, 282, 450, 466]
[100, 100, 130, 282, 450, 465]
[100, 100, 131, 282, 450, 464]
[100, 100, 132, 282, 450, 463]
[100, 10

In [20]:
def find_swaps(trades, target_sum, max_for_first, min_for_second):
    """
    Find all pairs in trades that sum to target_sum, where the first is less than max_for_first
    and the second is greater than or equal to min_for_second.
    """
    ans = []
    left, right = 0, len(trades) - 1
    while left < right:
        if trades[left] >= max_for_first or trades[right] < min_for_second:
            break  # No valid pairs if we violate min/max constraints
        current_sum = trades[left] + trades[right]
        if current_sum == target_sum:
            ans.append((trades[left], trades[right]))
            left += 1  # Move pointers to find next pair
            right -= 1
        elif current_sum < target_sum:
            left += 1
        else:  # current_sum > target_sum
            right -= 1
    return ans

def find_new(start, trades):
    ans = [start]
    n = len(start)
    
    for i in range(n - 1):
        for j in range(i + 1, n):
            # Setting bounds based on adjacent elements
            max_for_first = start[j] if j < n - 1 else float('inf')
            min_for_second = start[i] if i > 0 else float('-inf')
            target_sum = start[i] + start[j]

            swaps = find_swaps(trades, target_sum, max_for_first, min_for_second)
            
            for first, second in swaps:
                # Ensure the swapped numbers maintain the sorted order
                if first < max_for_first and second >= min_for_second:
                    new_combo = start.copy()
                    new_combo[i], new_combo[j] = first, second
                    # Sort the subarray if needed to maintain overall order
                    new_combo[i:j+1] = sorted(new_combo[i:j+1])
                    if new_combo not in ans:
                        ans.append(new_combo)
    return ans

# Example usage
start = [100, 100, 100, 282, 450, 495]
# Assuming trades is a sorted list of all candidate numbers for swapping
trades = sorted(set(start + [num for num in range(101, 494)])) 

new_combinations = find_new(start, trades)
for combo in new_combinations:
    print(combo)


[100, 100, 100, 282, 450, 495]
[100, 100, 101, 281, 450, 495]
[100, 100, 102, 280, 450, 495]
[100, 100, 103, 279, 450, 495]
[100, 100, 104, 278, 450, 495]
[100, 100, 105, 277, 450, 495]
[100, 100, 106, 276, 450, 495]
[100, 100, 107, 275, 450, 495]
[100, 100, 108, 274, 450, 495]
[100, 100, 109, 273, 450, 495]
[100, 100, 110, 272, 450, 495]
[100, 100, 111, 271, 450, 495]
[100, 100, 112, 270, 450, 495]
[100, 100, 113, 269, 450, 495]
[100, 100, 114, 268, 450, 495]
[100, 100, 115, 267, 450, 495]
[100, 100, 116, 266, 450, 495]
[100, 100, 117, 265, 450, 495]
[100, 100, 118, 264, 450, 495]
[100, 100, 119, 263, 450, 495]
[100, 100, 120, 262, 450, 495]
[100, 100, 121, 261, 450, 495]
[100, 100, 122, 260, 450, 495]
[100, 100, 123, 259, 450, 495]
[100, 100, 124, 258, 450, 495]
[100, 100, 125, 257, 450, 495]
[100, 100, 126, 256, 450, 495]
[100, 100, 127, 255, 450, 495]
[100, 100, 128, 254, 450, 495]
[100, 100, 129, 253, 450, 495]
[100, 100, 130, 252, 450, 495]
[100, 100, 131, 251, 450, 495]
[100, 10

In [23]:
tuples_combo = [tuple(c) for c in new_combinations]
len(new_combinations) == len(set(tuples_combo))

True